In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

# Verzeichnisse für Trainings- und Validierungsdaten
train_dir = 'images/archive/bloodcells_dataset/train'
valid_dir = 'images/archive/bloodcells_dataset/validation'

# ImageDataGenerator für Datenaugmentation und Normalisierung
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)

# Größe der Bilder
img_size = (224, 224)

# Batch-Größe
batch_size = 32

# Trainingsdaten-Generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Validierungsdaten-Generator
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Laden der vortrainierten ResNet50-Modellbasis ohne den Klassifikationskopf
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Einfügen eines eigenen Klassifikationskopfs
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='softmax'))

# Einfrieren der Gewichte der vortrainierten Modellbasis
base_model.trainable = False

# Kompilieren des Modells
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Anzahl der Epochen
epochs = 1

# Trainieren des Modells
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=valid_generator
)

# Auswertung des Modells auf den Validierungsdaten
eval_result = model.evaluate(valid_generator)
print("Loss auf den Validierungsdaten:", eval_result[0])
print("Genauigkeit auf den Validierungsdaten:", eval_result[1])

# Beispiel: Vorhersagen mit dem trainierten Modell
sample_images, sample_labels = next(iter(valid_generator))
predictions = model.predict(sample_images)


Found 13671 images belonging to 8 classes.
Found 3421 images belonging to 8 classes.
 14/428 [..............................] - ETA: 28:09 - loss: 2.3204 - accuracy: 0.1540